In [2]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
from pandas_datareader import data
import yfinance as yf

In [4]:
# input
symbols = ['AAPL','MSFT','AMD','NVDA','GE']
start = '2012-01-01'
end = '2019-09-11'

# Read data 
dataset = yf.download(symbols,start,end)['Close']

# View Columns
close = dataset.fillna(0)

[*********************100%***********************]  5 of 5 completed


In [38]:
cl = close.round(2)

In [39]:
cl

,AAPL,AMD,GE,MSFT,NVDA
Date,,,,,
2012-01-03,14.69,5.48,141.23,26.77,3.51
2012-01-04,14.77,5.46,142.77,27.40,3.55
2012-01-05,14.93,5.46,142.69,27.68,3.68
2012-01-06,15.09,5.43,143.46,28.11,3.63
2012-01-09,15.06,5.59,145.08,27.74,3.63
...,...,...,...,...,...
2019-09-04,52.30,30.95,70.40,137.63,42.19
2019-09-05,53.32,31.50,70.48,140.05,44.94
2019-09-06,53.31,30.56,69.68,139.10,44.66


In [18]:
n = 'MSFT'

In [51]:
cl.loc[cl['AMD'] == 5.48]['GE'] = 2

/Users/kexin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
close['AMD'] == 5.48

Date
2012-01-03    False
2012-01-04    False
2012-01-05    False
2012-01-06    False
2012-01-09    False
              ...  
2019-09-04    False
2019-09-05    False
2019-09-06    False
2019-09-09    False
2019-09-10    False
Name: AMD, Length: 1934, dtype: bool

In [27]:
close

,AAPL,AMD,GE,MSFT,NVDA
Date,,,,,
2012-01-03,14.686786,5.480000,141.230774,26.770000,3.510000
2012-01-04,14.765714,5.460000,142.769226,27.400000,3.550000
2012-01-05,14.929643,5.460000,142.692307,27.680000,3.677500
2012-01-06,15.085714,5.430000,143.461533,28.110001,3.635000
2012-01-09,15.061786,5.590000,145.076920,27.740000,3.635000
...,...,...,...,...,...
2019-09-04,52.297501,30.950001,70.400002,137.630005,42.189999
2019-09-05,53.320000,31.500000,70.480003,140.050003,44.935001
2019-09-06,53.314999,30.559999,69.680000,139.100006,44.662498


['G', 'E']

In [10]:
a = [1,2,3,4]
b = [1,2]
list(set(a)-set(b))

[3, 4]

In [13]:
a+b

[1, 2, 3, 4, 1, 2]

In [61]:
def best_combination(pnl_collated, stats, threshold):
    stats_sorted = stats.sort_values(by=['OS:Sharpe'], ascending = False)
    num_alphas = stats.shape[0]
    sorted_alphas = stats[['alpha_name']]
    curr_best_combi = [sorted_alphas[0]]
    count = 1
    curr_sharpe = stats.loc[stats['alpha_name'] == sorted_alphas[0]]['OS:Sharpe'].item()
    while count <= num_alphas:
        find_best = list(set(sorted_alphas)-set(curr_best_combi))
        temp = curr_best_combi.copy()
        for alpha in find_best:
            curr_pnl = pnl_collated[curr_best_combi + [alpha]].sum(axis=1)
            if get_sharpe(curr_pnl) > curr_sharpe and get_sharpe(curr_pnl) > threshold:
                curr_sharpe = get_sharpe(curr_pnl)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
            elif get_sharpe(curr_pnl) == curr_sharpe and get_turnover(positions) > curr_turnover:
                curr_turnover = get_turnover(positions)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
            elif get_sharpe(curr_pnl) == curr_sharpe and get_turnover(positions) == curr_turnover and get_mdd(curr_position) < curr_mdd:
                curr_mdd = get_mdd(curr_position)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
        curr_best_combi = temp
        count += 1
    best_sharpe = get_overall(curr_pnl)
    return curr_best_combi, curr_pnl, best_sharpe

In [60]:
def best_combo_of(pnl_collated, stats, threshold, num=2):
    tats_sorted = stats.sort_values(by=['OS:Sharpe'], ascending = False)
    sorted_alphas = stats[['alpha_name']]
    curr_best_combi = [sorted_alphas[0]]
    count = 1
    curr_sharpe = stats.loc[stats['alpha_name'] == sorted_alphas[0]]['OS:Sharpe'].item()
    while count <= num:
        find_best = list(set(sorted_alphas)-set(curr_best_combi))
        temp = curr_best_combi.copy()
        for alpha in find_best:
            curr_pnl = pnl_collated[curr_best_combi + [alpha]].sum(axis=1)
            if get_sharpe(curr_pnl) > curr_sharpe and get_sharpe(curr_pnl) > threshold:
                curr_sharpe = get_sharpe(curr_pnl)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
            elif get_sharpe(curr_pnl) == curr_sharpe and get_turnover(positions) > curr_turnover:
                curr_turnover = get_turnover(positions)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
            elif get_sharpe(curr_pnl) == curr_sharpe and get_turnover(positions) == curr_turnover and get_mdd(curr_position) < curr_mdd:
                curr_mdd = get_mdd(curr_position)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
        curr_best_combi = temp
        count += 1
    best_sharpe = get_overall(curr_pnl)
    return curr_best_combi, curr_pnl, best_sharpe

In [ ]:
def 